## Задачи исследования
1. Отличаются ли пелены от бинтов по параметрам?
2. Отличаются ли узкие, средние и широкие бинты друг от друга по параметрам?
3. Каков процент рогожки?
4. Какие группы тканей по качеству получаются?
5. Различаются ли нити основы и нити утка?
6. Какое количество (процент) тканей высокого качества имеют сбалансированное переплетение?
7. Какое количество (процент) тканей среднего качества имеет сбалансированное переплетение?
8. Какое количество (процент) тканей ниского качества имеет сбалансированное переплетение?
9. Каково соотношение сбалансированных тканей и тканей с преобладанием нитей основы?
10. Высчитать "коэффициент заполняемости" тканей
11. Разделить ткани по сбалансированности (сбалансированные - ±3 нити; ткани с преобладанием нитей основы); рассчитать процент каждой группы

In [1]:
import pandas
import pandas_profiling
import seaborn
import scipy.stats as stats
import matplotlib.pyplot as pyplot
%matplotlib inline

import modules.textiles as textile

In [2]:
textiles = textile.Textiles('dataframe.csv')
# outliers = textile.TextileOutliers(textiles)
# dataframe = textiles.dataframe()
# print(len(dataframe))

# # Ткани с переплетением типа "рогожка";
# baskets = dataframe[dataframe['weaving_technique'] == 'basket']

# # выборка узких бинтов;
# narrow_bands = dataframe[dataframe['type'] == 'n_band']

# # выборка средних бинтов;
# middle_bands = dataframe[dataframe['type'] == 'm_band']

# # выборка широких бинтов;
# wide_bands = dataframe[dataframe['type'] == 'w_band']

In [5]:
# report = pandas_profiling.ProfileReport(textiles.dataframe())
# report.get_description()

### Описание датафрейма
1. 'internal_storage' - номер по каталогу
2. 'grave' - номер могилы
3. 'warp_size' - размер изделия по основе
4. 'weft_size' - размер изделия по утку
5. 'type' - тип изделия
6. 'condition' - сохранность изделия
7. 'weaving_technique'
8. 'warp_material' - материал нитей основы
9. 'weft_material' - материал нитей утка
10. 'warp_dyed' - окрашивание нитей основы
11. 'weft_dyed' - окрашивание нитей утка
12. 'warp_twist' -  крутка нитей основы
13. 'weft_twist' - крутка нитей утка
14. 'warp_angle'
15. 'weft_angle'
16. 'warp_a' - толщина нитей основы (нижний предел)
17. 'warp_b' - толщина нитей основы (верхний предел)
18. 'weft_a' - толщина нитей утка (нижний предел)
19. 'weft_b' - толщина нитей утка (верхний предел)
20. 'warp_dens' - плотность по основе
21. 'weft_dens' - плотность по утку  
* * *
Числовые параметры - warp_size, wef_size, warp_a, warp_b, weft_a, weft_b, warp_mean, weft_mean, warp_dens, weft_dens

In [23]:
dataframe = textiles.dataframe()
# dataframe.head()
densities = dataframe[['warp_dens', 'weft_dens']]
densities_correlation = densities.corr()
print(densities_correlation)

           warp_dens  weft_dens
warp_dens   1.000000   0.531992
weft_dens   0.531992   1.000000


In [26]:
balanced = textiles.balanced()
unbalanced = textiles.unbalanced()
print('Сбалансированных тканей - ' + str(len(balanced)))
print('Несбалансированных тканей - ' + str(len(unbalanced)))

Сбалансированных тканей - 47
Несбалансированных тканей - 256
